In [ ]:
import os
import cv2

In [ ]:
import os
from PIL import Image
import numpy as np
from IPython.display import Image
import torch
import PIL

In [ ]:
dir_path="/content/drive/MyDrive/2기 딥러닝 프로젝트/train"
labels = ['Closed', 'Open']
x_data = []
y_data = []
for label in labels:
   path = os.path.join(dir_path, label)
   class_num = labels.index(label)
   import torchvision.transforms as transforms
   tf = transforms.ToTensor()
   for img in os.listdir(path): # closed 폴더 안에 _1.jpg, open 폴더 안에 1.jpg 
       paths = os.path.join(path, img)
       img =  PIL.Image.open(paths)
       re_img = img.resize((64,64))
       re_img = tf(re_img)
       re_img = re_img.tolist()
       if len(re_img) == 3:
         x_data.append(re_img)
         y_data.append(class_num)

x_data = torch.tensor(x_data)
y_data = torch.tensor(y_data)

자꾸 이런 expected sequence of length 3 at dim 1 (got 4) 오류가 나서

if len(re_img) == 3: 코드를 추가해주었고 이걸 하고 나니까 원래는 x shape이 1451이었는데 1449로 줄어들었다 아마 2개의 x_data의 shape이 다른 사진들과 달랐던 것 같다

In [ ]:
x_data.shape

torch.Size([1449, 3, 64, 64])

In [ ]:
y_data.shape

torch.Size([1449])

In [ ]:
y_data = np.array(y_data)
y_data = torch.tensor(y_data)
y_data.shape

torch.Size([1449])

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

In [ ]:
x_train.shape

torch.Size([1159, 3, 64, 64])

In [ ]:
y_train.shape

torch.Size([1159])

## **train**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import torch.optim as optim

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(4096, 512) 
        self.fc2 = nn.Linear(512, 1)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2)
        x = x.reshape(-1, 4096)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

x = torch.ones([1,3,64,64])
conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
output1 = conv1(x)
print(output1.shape)
x1 = F.max_pool2d(F.relu(output1),2)
print(x1.shape)
conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
output2 = conv2(x1)
print(output2.shape)
x2 = F.max_pool2d(F.relu(output2), 2)
print(x2.shape)
conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
output3 = conv3(x2)
print(output3.shape)
x3 = F.max_pool2d(F.relu(output3), 2)
print(x3.shape)

torch.Size([1, 16, 64, 64])
torch.Size([1, 16, 32, 32])
torch.Size([1, 32, 32, 32])
torch.Size([1, 32, 16, 16])
torch.Size([1, 64, 16, 16])
torch.Size([1, 64, 8, 8])


In [ ]:
model = Net()
x = torch.ones(1, 3,64,64)
x = x.float()
output = model(x)
output.shape

torch.Size([1, 1])

In [ ]:
model = Net()
model.to('cuda')
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

epochs = 30

In [ ]:
def accuracy(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum / y_test.shape[0]
    acc = torch.round(acc * 100)

    return acc

In [ ]:
for epoch in range(epochs):
    running_loss = 0.0
    running_acc = 0.0

    model.train()

    for i,data in enumerate(x_train, 0):
        optimizer.zero_grad()

        data = data.to('cuda')
        labels = y_train.to('cuda')

        k = torch.tensor(labels[i])
        k = k.view(1,-1)
        k = k.float()
        k = k.to('cuda')
        outputs = model(data)
        #print("intpus_1 shape: ",input_1.shape)
        #print("outputs shape: ",outputs.shape)
        #print("labels shape",labels.shape)
         #labels = labels.view(len(labels),-1)
        loss = criterion(outputs, k)
        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_acc += accuracy(outputs, k)

        if i % 80 == 79:
            print('epoch: [%d/%d] train_loss: %.5f train_acc: %.5f' % (
                epoch + 1, epochs, running_loss / 80, running_acc / 80))
            running_loss = 0.0

print("learning finish")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


epoch: [1/50] train_loss: 0.69876 train_acc: 48.75000
epoch: [1/50] train_loss: 0.69525 train_acc: 97.50000
epoch: [1/50] train_loss: 0.68318 train_acc: 156.25000
epoch: [1/50] train_loss: 0.67837 train_acc: 220.00000
epoch: [1/50] train_loss: 0.65744 train_acc: 287.50000
epoch: [1/50] train_loss: 0.56549 train_acc: 368.75000
epoch: [1/50] train_loss: 0.46055 train_acc: 451.25000
epoch: [1/50] train_loss: 0.31957 train_acc: 540.00000
epoch: [1/50] train_loss: 0.25567 train_acc: 630.00000
epoch: [1/50] train_loss: 0.22548 train_acc: 722.50000
epoch: [1/50] train_loss: 0.13178 train_acc: 820.00000
epoch: [1/50] train_loss: 0.09688 train_acc: 916.25000
epoch: [1/50] train_loss: 0.16200 train_acc: 1007.50000
epoch: [1/50] train_loss: 0.19212 train_acc: 1098.75000
epoch: [2/50] train_loss: 0.18044 train_acc: 96.25000
epoch: [2/50] train_loss: 0.30371 train_acc: 185.00000
epoch: [2/50] train_loss: 0.15079 train_acc: 278.75000
epoch: [2/50] train_loss: 0.17988 train_acc: 372.50000
epoch: [2/5

# **test**

In [ ]:
with torch.no_grad():
    total_acc = 0.0
    acc = 0.0
    data = x_test.to('cuda')
    labels = y_test.to('cuda')

    for i,data in enumerate(x_test, 0):
        data = data.to('cuda')
        labels = y_test.to('cuda')
        k = torch.tensor(labels[i])
        k = k.view(1,-1)
        k = k.float()
        k = k.to('cuda')
        outputs = model(data)
        acc = accuracy(outputs, k)
        total_acc += acc

        count = i

    print('avarage acc: %.5f' % (total_acc/count),'%')

print('test finish!')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


avarage acc: 97.92388 %
test finish!


In [ ]:
def predict(pred):
  pred = pred
  outputs = model(pred)
  pred_tag = torch.round(torch.sigmoid(outputs))
  #pred_tag = torch.sigmoid(outputs)
  return pred_tag

In [ ]:
def predicts(pred):
  pred = pred
  outputs = model(pred)
  pred_tag = torch.sigmoid(outputs)
  return pred_tag

# **test해보기**

In [ ]:
image = PIL.Image.open('/content/눈사진.jpg')
img =image.resize((64,64))
plt.imshow(img)
re_img = tf(img) # 해주면 image의 shape는 (3,64,64) 이된다
re_img = re_img.to('cuda')
pred_tag = predict(re_img)
print("눈사진: ", pred_tag)

prediction = predicts(re_img)
 
if prediction < 0.5: # 눈이 감으면
   text = "Closed ({:.2f}%)".format((1 - prediction[0][0])*100)
   print(text)

else: #눈을 뜨면
   text = "Open ({:.2f}%)".format(prediction[0][0]*100)
   print(text)

FileNotFoundError: ignored

In [ ]:
image2 = PIL.Image.open('/content/눈감음.jpg')
img2 =image2.resize((64,64))
plt.imshow(img2)
re_img2 = tf(img2) # 해주면 image의 shape는 (3,64,64) 이된다
re_img2 = re_img2.to('cuda')
pred_tag2 = predict(re_img2)
print("눈사진: ", pred_tag2)

prediction = predicts(re_img2)
 
if prediction < 0.5: # 눈이 감으면
   text = "Closed ({:.2f}%)".format((1 - prediction[0][0])*100)
   print(text)

else: #눈을 뜨면
   text = "Open ({:.2f}%)".format(prediction[0][0]*100)
   print(text)

# 모델 수정해보기 1 : layer를 늘려보자

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1) #  추가시켜줌
        self.fc1 = nn.Linear(2048, 512) 
        self.fc2 = nn.Linear(512, 1)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2)
        x = F.max_pool2d(F.relu(self.conv4(x)), 2)
        x = x.reshape(-1, 2048)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = Net()
model.to('cuda')
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

epochs = 50

In [ ]:
for epoch in range(epochs):
    running_loss = 0.0
    running_acc = 0.0

    model.train()

    for i,data in enumerate(x_train, 0):
        optimizer.zero_grad()

        data = data.to('cuda')
        labels = y_train.to('cuda')

        k = torch.tensor(labels[i])
        k = k.view(1,-1)
        k = k.float()
        k = k.to('cuda')
        outputs = model(data)
        #print("intpus_1 shape: ",input_1.shape)
        #print("outputs shape: ",outputs.shape)
        #print("labels shape",labels.shape)
         #labels = labels.view(len(labels),-1)
        loss = criterion(outputs, k)
        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_acc += accuracy(outputs, k)

        if i % 80 == 79:
            print('epoch: [%d/%d] train_loss: %.5f train_acc: %.5f' % (
                epoch + 1, epochs, running_loss / 80, running_acc / 80))
            running_loss = 0.0

print("learning finish")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


epoch: [1/50] train_loss: 0.69531 train_acc: 47.50000
epoch: [1/50] train_loss: 0.69258 train_acc: 96.25000
epoch: [1/50] train_loss: 0.64048 train_acc: 165.00000
epoch: [1/50] train_loss: 0.49626 train_acc: 245.00000
epoch: [1/50] train_loss: 0.27600 train_acc: 332.50000
epoch: [1/50] train_loss: 0.27318 train_acc: 423.75000
epoch: [1/50] train_loss: 0.18283 train_acc: 517.50000
epoch: [1/50] train_loss: 0.22324 train_acc: 607.50000
epoch: [1/50] train_loss: 0.14839 train_acc: 700.00000
epoch: [1/50] train_loss: 0.21731 train_acc: 792.50000
epoch: [1/50] train_loss: 0.11580 train_acc: 890.00000
epoch: [1/50] train_loss: 0.07603 train_acc: 988.75000
epoch: [1/50] train_loss: 0.11573 train_acc: 1083.75000
epoch: [1/50] train_loss: 0.20375 train_acc: 1176.25000
epoch: [2/50] train_loss: 0.17389 train_acc: 95.00000
epoch: [2/50] train_loss: 0.31015 train_acc: 183.75000
epoch: [2/50] train_loss: 0.14034 train_acc: 280.00000
epoch: [2/50] train_loss: 0.16060 train_acc: 373.75000
epoch: [2/5

In [ ]:
with torch.no_grad():
    total_acc = 0.0
    acc = 0.0
    data = x_test.to('cuda')
    labels = y_test.to('cuda')

    for i,data in enumerate(x_test, 0):
        data = data.to('cuda')
        labels = y_test.to('cuda')
        k = torch.tensor(labels[i])
        k = k.view(1,-1)
        k = k.float()
        k = k.to('cuda')
        outputs = model(data)
        acc = accuracy(outputs, k)
        total_acc += acc

        count = i

    print('avarage acc: %.5f' % (total_acc/count),'%')

print('test finish!')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


avarage acc: 98.61592 %
test finish!


### 층을 하나 더 늘리니 성능이  1% 정도 더 좋아졌다

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
 
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1) # 이번에는 필터의 개수를 2배로 늘려줬다
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1) 
        self.fc1 = nn.Linear(4096, 512)
        self.fc2 = nn.Linear(512, 1)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2)
        x = F.max_pool2d(F.relu(self.conv4(x)), 2)
        x = x.reshape(-1, 4096)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = Net()
model.to('cuda')
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

epochs = 50

In [ ]:
for epoch in range(epochs):
    running_loss = 0.0
    running_acc = 0.0

    model.train()

    for i,data in enumerate(x_train, 0):
        optimizer.zero_grad()

        data = data.to('cuda')
        labels = y_train.to('cuda')

        k = torch.tensor(labels[i])
        k = k.view(1,-1)
        k = k.float()
        k = k.to('cuda')
        outputs = model(data)
        #print("intpus_1 shape: ",input_1.shape)
        #print("outputs shape: ",outputs.shape)
        #print("labels shape",labels.shape)
         #labels = labels.view(len(labels),-1)
        loss = criterion(outputs, k)
        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_acc += accuracy(outputs, k)

        if i % 80 == 79:
            print('epoch: [%d/%d] train_loss: %.5f train_acc: %.5f' % (
                epoch + 1, epochs, running_loss / 80, running_acc / 80))
            running_loss = 0.0

print("learning finish")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


epoch: [1/50] train_loss: 0.69522 train_acc: 53.75000
epoch: [1/50] train_loss: 0.69753 train_acc: 96.25000
epoch: [1/50] train_loss: 0.63556 train_acc: 153.75000
epoch: [1/50] train_loss: 0.45697 train_acc: 238.75000
epoch: [1/50] train_loss: 0.31868 train_acc: 325.00000
epoch: [1/50] train_loss: 0.34670 train_acc: 412.50000
epoch: [1/50] train_loss: 0.20587 train_acc: 506.25000
epoch: [1/50] train_loss: 0.23118 train_acc: 598.75000
epoch: [1/50] train_loss: 0.16672 train_acc: 688.75000
epoch: [1/50] train_loss: 0.24422 train_acc: 782.50000
epoch: [1/50] train_loss: 0.11755 train_acc: 880.00000
epoch: [1/50] train_loss: 0.07984 train_acc: 976.25000
epoch: [1/50] train_loss: 0.10404 train_acc: 1072.50000
epoch: [1/50] train_loss: 0.26656 train_acc: 1166.25000
epoch: [2/50] train_loss: 0.21431 train_acc: 93.75000
epoch: [2/50] train_loss: 0.30619 train_acc: 178.75000
epoch: [2/50] train_loss: 0.19386 train_acc: 271.25000
epoch: [2/50] train_loss: 0.16252 train_acc: 363.75000
epoch: [2/5

In [ ]:
with torch.no_grad():
    total_acc = 0.0
    acc = 0.0
    data = x_test.to('cuda')
    labels = y_test.to('cuda')

    for i,data in enumerate(x_test, 0):
        data = data.to('cuda')
        labels = y_test.to('cuda')
        k = torch.tensor(labels[i])
        k = k.view(1,-1)
        k = k.float()
        k = k.to('cuda')
        outputs = model(data)
        acc = accuracy(outputs, k)
        total_acc += acc

        count = i

    print('avarage acc: %.5f' % (total_acc/count),'%')

print('test finish!')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


avarage acc: 98.96194 %
test finish!


### 정확도가 살짝오름

In [ ]:
#learning_rate: 기울기에 곱해서 얼만큼 이동할지 결정해줌

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
 
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1) # 이번에는 필터의 개수를 2배로 늘려줬다
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1) 
        self.fc1 = nn.Linear(4096, 512)
        self.fc2 = nn.Linear(512, 1)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2)
        x = F.max_pool2d(F.relu(self.conv4(x)), 2)
        x = x.reshape(-1, 4096)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = Net()
model.to('cuda')
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) # 0.0001에서 0.001로 해줌 한번 바꿀때 이전보다 값이 더 많이 바뀌게 될 거다 

epochs = 50

In [ ]:
for epoch in range(epochs):
    running_loss = 0.0
    running_acc = 0.0

    model.train()

    for i,data in enumerate(x_train, 0):
        optimizer.zero_grad()

        data = data.to('cuda')
        labels = y_train.to('cuda')

        k = torch.tensor(labels[i])
        k = k.view(1,-1)
        k = k.float()
        k = k.to('cuda')
        outputs = model(data)
        #print("intpus_1 shape: ",input_1.shape)
        #print("outputs shape: ",outputs.shape)
        #print("labels shape",labels.shape)
         #labels = labels.view(len(labels),-1)
        loss = criterion(outputs, k)
        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_acc += accuracy(outputs, k)

        if i % 80 == 79:
            print('epoch: [%d/%d] train_loss: %.5f train_acc: %.5f' % (
                epoch + 1, epochs, running_loss / 80, running_acc / 80))
            running_loss = 0.0

print("learning finish")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


epoch: [1/50] train_loss: 0.72822 train_acc: 45.00000
epoch: [1/50] train_loss: 0.69280 train_acc: 100.00000
epoch: [1/50] train_loss: 0.69423 train_acc: 141.25000
epoch: [1/50] train_loss: 0.69345 train_acc: 193.75000
epoch: [1/50] train_loss: 0.69428 train_acc: 237.50000
epoch: [1/50] train_loss: 0.69290 train_acc: 292.50000
epoch: [1/50] train_loss: 0.69472 train_acc: 338.75000
epoch: [1/50] train_loss: 0.69080 train_acc: 398.75000
epoch: [1/50] train_loss: 0.69351 train_acc: 450.00000
epoch: [1/50] train_loss: 0.69537 train_acc: 496.25000
epoch: [1/50] train_loss: 0.69399 train_acc: 545.00000
epoch: [1/50] train_loss: 0.69614 train_acc: 595.00000
epoch: [1/50] train_loss: 0.69865 train_acc: 648.75000
epoch: [1/50] train_loss: 0.69626 train_acc: 695.00000
epoch: [2/50] train_loss: 0.85056 train_acc: 56.25000
epoch: [2/50] train_loss: 0.79501 train_acc: 122.50000
epoch: [2/50] train_loss: 0.63800 train_acc: 202.50000
epoch: [2/50] train_loss: 0.64248 train_acc: 272.50000
epoch: [2/50

In [ ]:
with torch.no_grad():
    total_acc = 0.0
    acc = 0.0
    data = x_test.to('cuda')
    labels = y_test.to('cuda')

    for i,data in enumerate(x_test, 0):
        data = data.to('cuda')
        labels = y_test.to('cuda')
        k = torch.tensor(labels[i])
        k = k.view(1,-1)
        k = k.float()
        k = k.to('cuda')
        outputs = model(data)
        acc = accuracy(outputs, k)
        total_acc += acc

        count = i

    print('avarage acc: %.5f' % (total_acc/count),'%')

print('test finish!')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


avarage acc: 99.65398 %
test finish!


### 예측성능이 제일 좋아진 것 같다

### 커널 사이즈를 바꿔보자

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=1)
        self.fc1 = nn.Linear(2304, 512) 
        self.fc2 = nn.Linear(512, 1)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2)
        x = x.reshape(-1, 2304)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = Net()
model.to('cuda')
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

epochs = 30

In [ ]:
for epoch in range(epochs):
    running_loss = 0.0
    running_acc = 0.0

    model.train()

    for i,data in enumerate(x_train, 0):
        optimizer.zero_grad()

        data = data.to('cuda')
        labels = y_train.to('cuda')

        k = torch.tensor(labels[i])
        k = k.view(1,-1)
        k = k.float()
        k = k.to('cuda')
        outputs = model(data)
        #print("intpus_1 shape: ",input_1.shape)
        #print("outputs shape: ",outputs.shape)
        #print("labels shape",labels.shape)
         #labels = labels.view(len(labels),-1)
        loss = criterion(outputs, k)
        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_acc += accuracy(outputs, k)

        if i % 1120 == 1119:
            print('epoch: [%d/%d] train_loss: %.3f train_acc: %.3f' % (
                epoch + 1, epochs, running_loss / 1120, running_acc / 1120))
            running_loss = 0.0

print("learning finish")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


epoch: [1/30] train_loss: 0.300 train_acc: 85.982
epoch: [2/30] train_loss: 0.144 train_acc: 94.911
epoch: [3/30] train_loss: 0.094 train_acc: 96.518
epoch: [4/30] train_loss: 0.065 train_acc: 97.946
epoch: [5/30] train_loss: 0.059 train_acc: 98.482
epoch: [6/30] train_loss: 0.040 train_acc: 98.482
epoch: [7/30] train_loss: 0.021 train_acc: 99.286
epoch: [8/30] train_loss: 0.030 train_acc: 98.750
epoch: [9/30] train_loss: 0.026 train_acc: 99.196
epoch: [10/30] train_loss: 0.022 train_acc: 99.375
epoch: [11/30] train_loss: 0.016 train_acc: 99.196
epoch: [12/30] train_loss: 0.028 train_acc: 99.464
epoch: [13/30] train_loss: 0.007 train_acc: 99.732
epoch: [14/30] train_loss: 0.007 train_acc: 99.911
epoch: [15/30] train_loss: 0.007 train_acc: 99.732
epoch: [16/30] train_loss: 0.004 train_acc: 99.911
epoch: [17/30] train_loss: 0.017 train_acc: 99.643
epoch: [18/30] train_loss: 0.010 train_acc: 99.732
epoch: [19/30] train_loss: 0.001 train_acc: 100.000
epoch: [20/30] train_loss: 0.000 train_

In [ ]:
with torch.no_grad():
    total_acc = 0.0
    acc = 0.0
    data = x_test.to('cuda')
    labels = y_test.to('cuda')

    for i,data in enumerate(x_test, 0):
        data = data.to('cuda')
        labels = y_test.to('cuda')
        k = torch.tensor(labels[i])
        k = k.view(1,-1)
        k = k.float()
        k = k.to('cuda')
        outputs = model(data)
        acc = accuracy(outputs, k)
        total_acc += acc

        count = i

    print('avarage acc: %.5f' % (total_acc/count),'%')

print('test finish!')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


avarage acc: 97.92388 %
test finish!


### 패딩의 여부

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(1024, 512) 
        self.fc2 = nn.Linear(512, 1)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2)
        x = x.reshape(-1, 1024)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = Net()
model.to('cuda')
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

epochs = 30

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
x = torch.ones([1,3,64,64])
conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=1)
output1 = conv1(x)
print(output1.shape)
x1 = F.max_pool2d(F.relu(output1),2)
print(x1.shape)
conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=1)
output2 = conv2(x1)
print(output2.shape)
x2 = F.max_pool2d(F.relu(output2), 2)
print(x2.shape)
conv3 = nn.Conv2d(32, 64, kernel_size=5, stride=1)
output3 = conv3(x2)
print(output3.shape)
x3 = F.max_pool2d(F.relu(output3), 2)
print(x3.shape)

torch.Size([1, 16, 60, 60])
torch.Size([1, 16, 30, 30])
torch.Size([1, 32, 26, 26])
torch.Size([1, 32, 13, 13])
torch.Size([1, 64, 9, 9])
torch.Size([1, 64, 4, 4])


In [ ]:
for epoch in range(epochs):
    running_loss = 0.0
    running_acc = 0.0

    model.train()

    for i,data in enumerate(x_train, 0):
        optimizer.zero_grad()

        data = data.to('cuda')
        labels = y_train.to('cuda')

        k = torch.tensor(labels[i])
        k = k.view(1,-1)
        k = k.float()
        k = k.to('cuda')
        outputs = model(data)
        #print("intpus_1 shape: ",input_1.shape)
        #print("outputs shape: ",outputs.shape)
        #print("labels shape",labels.shape)
         #labels = labels.view(len(labels),-1)
        loss = criterion(outputs, k)
        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_acc += accuracy(outputs, k)

        if i % 1120 == 1119:
            print('epoch: [%d/%d] train_loss: %.3f train_acc: %.3f' % (
                epoch + 1, epochs, running_loss / 1120, running_acc / 1120))
            running_loss = 0.0

print("learning finish")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


epoch: [1/30] train_loss: 0.294 train_acc: 86.161
epoch: [2/30] train_loss: 0.132 train_acc: 95.625
epoch: [3/30] train_loss: 0.094 train_acc: 96.607
epoch: [4/30] train_loss: 0.077 train_acc: 96.875
epoch: [5/30] train_loss: 0.066 train_acc: 97.589
epoch: [6/30] train_loss: 0.046 train_acc: 98.036
epoch: [7/30] train_loss: 0.034 train_acc: 98.750
epoch: [8/30] train_loss: 0.023 train_acc: 99.107
epoch: [9/30] train_loss: 0.039 train_acc: 98.750
epoch: [10/30] train_loss: 0.020 train_acc: 99.286
epoch: [11/30] train_loss: 0.015 train_acc: 99.643
epoch: [12/30] train_loss: 0.015 train_acc: 99.286
epoch: [13/30] train_loss: 0.021 train_acc: 99.375
epoch: [14/30] train_loss: 0.005 train_acc: 100.000
epoch: [15/30] train_loss: 0.005 train_acc: 99.911
epoch: [16/30] train_loss: 0.009 train_acc: 99.821
epoch: [17/30] train_loss: 0.011 train_acc: 99.554
epoch: [18/30] train_loss: 0.004 train_acc: 99.821
epoch: [19/30] train_loss: 0.016 train_acc: 99.643
epoch: [20/30] train_loss: 0.001 train_

In [ ]:
with torch.no_grad():
    total_acc = 0.0
    acc = 0.0
    data = x_test.to('cuda')
    labels = y_test.to('cuda')

    for i,data in enumerate(x_test, 0):
        data = data.to('cuda')
        labels = y_test.to('cuda')
        k = torch.tensor(labels[i])
        k = k.view(1,-1)
        k = k.float()
        k = k.to('cuda')
        outputs = model(data)
        acc = accuracy(outputs, k)
        total_acc += acc

        count = i

    print('avarage acc: %.5f' % (total_acc/count),'%')

print('test finish!')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


avarage acc: 98.96194 %
test finish!


###**yawn/ no yawn**

In [ ]:
dir_path="/content/drive/MyDrive/2기 딥러닝 프로젝트/train"
labels = ['no_yawn', 'yawn']
x_data = []
y_data = []
for label in labels:
   path = os.path.join(dir_path, label)
   class_num = labels.index(label)
   import torchvision.transforms as transforms
   tf = transforms.ToTensor()
   for img in os.listdir(path): # closed 폴더 안에 _1.jpg, open 폴더 안에 1.jpg 
       paths = os.path.join(path, img)
       img =  PIL.Image.open(paths)
       re_img = img.resize((64,64))
       re_img = tf(re_img)
       re_img = re_img.tolist()
       if len(re_img) == 3:
         x_data.append(re_img)
         y_data.append(class_num)

x_data = torch.tensor(x_data)
y_data = torch.tensor(y_data)

In [ ]:
y_data.shape

torch.Size([1448])

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

In [ ]:
for epoch in range(epochs):
    running_loss = 0.0
    running_acc = 0.0

    model.train()

    for i,data in enumerate(x_train, 0):
        optimizer.zero_grad()

        data = data.to('cuda')
        labels = y_train.to('cuda')

        k = torch.tensor(labels[i])
        k = k.view(1,-1)
        k = k.float()
        k = k.to('cuda')
        outputs = model(data)
        #print("intpus_1 shape: ",input_1.shape)
        #print("outputs shape: ",outputs.shape)
        #print("labels shape",labels.shape)
         #labels = labels.view(len(labels),-1)
        loss = criterion(outputs, k)
        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_acc += accuracy(outputs, k)

        if i % 1120 == 1119:
            print('epoch: [%d/%d] train_loss: %.3f train_acc: %.3f' % (
                epoch + 1, epochs, running_loss / 1120, running_acc / 1120))
            running_loss = 0.0

print("learning finish")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


epoch: [1/30] train_loss: 0.620 train_acc: 61.607
epoch: [2/30] train_loss: 0.566 train_acc: 67.054
epoch: [3/30] train_loss: 0.551 train_acc: 69.196
epoch: [4/30] train_loss: 0.532 train_acc: 72.232
epoch: [5/30] train_loss: 0.508 train_acc: 74.464
epoch: [6/30] train_loss: 0.480 train_acc: 77.232
epoch: [7/30] train_loss: 0.449 train_acc: 78.839
epoch: [8/30] train_loss: 0.416 train_acc: 81.518
epoch: [9/30] train_loss: 0.380 train_acc: 83.482
epoch: [10/30] train_loss: 0.341 train_acc: 85.625
epoch: [11/30] train_loss: 0.298 train_acc: 88.125
epoch: [12/30] train_loss: 0.256 train_acc: 90.089
epoch: [13/30] train_loss: 0.216 train_acc: 91.786
epoch: [14/30] train_loss: 0.179 train_acc: 93.929
epoch: [15/30] train_loss: 0.145 train_acc: 95.357
epoch: [16/30] train_loss: 0.118 train_acc: 96.696
epoch: [17/30] train_loss: 0.098 train_acc: 97.054
epoch: [18/30] train_loss: 0.080 train_acc: 97.679
epoch: [19/30] train_loss: 0.066 train_acc: 98.214
epoch: [20/30] train_loss: 0.057 train_a

In [ ]:
with torch.no_grad():
    total_acc = 0.0
    acc = 0.0
    data = x_test.to('cuda')
    labels = y_test.to('cuda')

    for i,data in enumerate(x_test, 0):
        data = data.to('cuda')
        labels = y_test.to('cuda')
        k = torch.tensor(labels[i])
        k = k.view(1,-1)
        k = k.float()
        k = k.to('cuda')
        outputs = model(data)
        acc = accuracy(outputs, k)
        total_acc += acc

        count = i

    print('avarage acc: %.5f' % (total_acc/count),'%')

print('test finish!')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


avarage acc: 94.80969 %
test finish!
